In [ ]:
# import numpy as np
# import matplotlib.pyplot as plt
# import matplotlib.patches as patches
# from collections import defaultdict
# import time

# plt.rcParams.update({
#     "font.size": 11,
#     "font.family": "serif",
#     "axes.titlesize": 12,
#     "axes.labelsize": 11
# })

# # ============================================================
# # 1. IMAGE LOADING
# # ============================================================

# def load_grayscale_image(path):
#     with open(path, 'rb') as f:
#         _ = f.read(512)
#         img = np.fromfile(f, dtype=np.uint8)
#     return img.reshape((512, 512))


# # ============================================================
# # 2. THRESHOLDING
# # ============================================================

# def generate_binary_mask(img, T=128):
#     binary = (img > T).astype(np.uint8)

#     # Invert if foreground dominates
#     if np.sum(binary) > (binary.size // 2):
#         binary = 1 - binary

#     return binary


# # ============================================================
# # 3. ITERATIVE 4-CONNECTED CCL
# # ============================================================

# def label_components_4connect(binary_img):
#     rows, cols = binary_img.shape
#     labels = np.zeros((rows, cols), dtype=int)
#     current_label = 1
#     equivalences = defaultdict(set)

#     for i in range(rows):
#         for j in range(cols):
#             if binary_img[i, j] == 1:

#                 neighbors = []

#                 if i > 0 and labels[i-1, j] > 0:
#                     neighbors.append(labels[i-1, j])

#                 if j > 0 and labels[i, j-1] > 0:
#                     neighbors.append(labels[i, j-1])

#                 if not neighbors:
#                     labels[i, j] = current_label
#                     current_label += 1

#                 elif len(neighbors) == 1:
#                     labels[i, j] = neighbors[0]

#                 else:
#                     min_label = min(neighbors)
#                     labels[i, j] = min_label
#                     for lab in neighbors:
#                         if lab != min_label:
#                             equivalences[min_label].add(lab)
#                             equivalences[lab].add(min_label)

#     # Resolve equivalences
#     label_map = {}
#     visited = set()

#     def dfs(label, root):
#         stack = [label]
#         while stack:
#             l = stack.pop()
#             if l not in visited:
#                 visited.add(l)
#                 label_map[l] = root
#                 stack.extend(equivalences[l])

#     for lab in range(1, current_label):
#         if lab not in visited:
#             dfs(lab, lab)

#     for i in range(rows):
#         for j in range(cols):
#             if labels[i, j] > 0:
#                 labels[i, j] = label_map[labels[i, j]]

#     return labels


# # ============================================================
# # 4. FEATURE EXTRACTION (WITH ECCENTRICITY)
# # ============================================================

# def extract_geometric_descriptors(labels, min_size):

#     descriptors = {}
#     unique_labels = np.unique(labels)
#     unique_labels = unique_labels[unique_labels != 0]

#     for lab in unique_labels:

#         coords = np.argwhere(labels == lab)
#         area = len(coords)

#         if area < min_size:
#             continue

#         y = coords[:, 0]
#         x = coords[:, 1]

#         xc = np.mean(x)
#         yc = np.mean(y)

#         min_x, max_x = np.min(x), np.max(x)
#         min_y, max_y = np.min(y), np.max(y)

#         x_prime = x - xc
#         y_prime = y - yc

#         a = np.sum(x_prime**2)
#         b = 2 * np.sum(x_prime * y_prime)
#         c = np.sum(y_prime**2)

#         theta = 0.5 * np.arctan2(b, (a - c))

#         term = np.sqrt((a - c)**2 + b**2)
#         Imax = (a + c + term) / 2
#         Imin = (a + c - term) / 2

#         elongation = Imax / Imin if Imin != 0 else 0

#         # ✅ ECCENTRICITY (Required by assignment)
#         eccentricity = np.sqrt(1 - (Imin / Imax)) if Imax != 0 else 0

#         # 8-connected boundary approximation
#         perimeter = 0
#         for py, px in coords:
#             neighbors = 0
#             for dy in [-1, 0, 1]:
#                 for dx in [-1, 0, 1]:
#                     if dy == 0 and dx == 0:
#                         continue
#                     ny, nx = py + dy, px + dx
#                     if 0 <= ny < labels.shape[0] and 0 <= nx < labels.shape[1]:
#                         if labels[ny, nx] == lab:
#                             neighbors += 1
#             if neighbors < 8:
#                 perimeter += 1

#         compactness = (perimeter**2) / area

#         descriptors[lab] = {
#             "area": area,
#             "centroid": (xc, yc),
#             "bbox": (min_x, min_y, max_x, max_y),
#             "orientation": theta,
#             "elongation": elongation,
#             "eccentricity": eccentricity,   # Added
#             "perimeter": perimeter,
#             "compactness": compactness
#         }

#     return descriptors


# # ============================================================
# # 5. VISUALIZATION
# # ============================================================
# # ============================================================
# # 5A. APA 7 STYLE TABLE RENDERING
# # ============================================================

# def render_feature_table(descriptors, size):

#     # Use APA-style serif font
#     plt.rcParams.update({
#         "font.family": "serif",
#         "font.serif": ["Times New Roman"],
#         "font.size": 11
#     })

#     table_data = []

#     for lab, comp in descriptors.items():

#         xc, yc = comp["centroid"]

#         table_data.append([
#             lab,
#             comp["area"],
#             f"({xc:.2f}, {yc:.2f})",
#             f"{comp['eccentricity']:.4f}",
#             f"{comp['compactness']:.2f}"
#         ])

#     column_labels = [
#         "ID",
#         "Area",
#         "Centroid (x, y)",
#         "Eccentricity",
#         "Compactness"
#     ]

#     fig, ax = plt.subplots(figsize=(9, 2 + 0.5 * len(table_data)))
#     ax.axis('off')

#     table = ax.table(
#         cellText=table_data,
#         colLabels=column_labels,
#         loc='center',
#         cellLoc='center'
#     )

#     table.auto_set_font_size(False)
#     table.set_fontsize(11)
#     table.scale(1, 1.4)

#     # Light border styling (APA clean look)
#     for key, cell in table.get_celld().items():
#         cell.set_linewidth(0.5)

#     plt.title(f"Table 1\nConnected Component Features (Size ≥ {size})",
#               fontsize=12, pad=15)

#     plt.tight_layout()
#     plt.savefig(f"CCL_Table_Size_{size}.png", dpi=600, bbox_inches='tight')
#     plt.show()


# def render_results(original, binary, filtered_labels, descriptors, size):

#     fig, axes = plt.subplots(2, 2, figsize=(12, 10))

#     axes[0, 0].imshow(original, cmap='gray')
#     axes[0, 0].set_title("Original Image")
#     axes[0, 0].axis('off')

#     axes[0, 1].imshow(binary, cmap='gray', vmin=0, vmax=1)
#     axes[0, 1].set_title("Binary Mask")
#     axes[0, 1].axis('off')

#     colored = np.zeros((*filtered_labels.shape, 3))

#     np.random.seed(42)
#     for lab in descriptors.keys():
#         color = np.random.rand(3)
#         colored[filtered_labels == lab] = color

#     axes[1, 0].imshow(colored)
#     axes[1, 0].set_title("Filtered Connected Components")
#     axes[1, 0].axis('off')

#     axes[1, 1].imshow(binary, cmap='gray', vmin=0, vmax=1)

#     for lab, comp in descriptors.items():
#         xc, yc = comp["centroid"]
#         min_x, min_y, max_x, max_y = comp["bbox"]

#         rect = patches.Rectangle(
#             (min_x, min_y),
#             max_x - min_x,
#             max_y - min_y,
#             linewidth=1.2,
#             edgecolor='cyan',
#             facecolor='none'
#         )
#         axes[1, 1].add_patch(rect)

#         length = 40
#         theta = comp["orientation"]
#         x2 = xc + length * np.cos(theta)
#         y2 = yc + length * np.sin(theta)
#         axes[1, 1].plot([xc, x2], [yc, y2], 'yellow', linewidth=1.5)

#         axes[1, 1].text(xc, yc, str(lab),
#                         color='white',
#                         fontsize=8,
#                         ha='center',
#                         va='center',
#                         bbox=dict(facecolor='black', alpha=0.6, pad=1))

#     axes[1, 1].set_title("Feature Overlay")
#     axes[1, 1].axis('off')

#     plt.tight_layout()
#     plt.savefig(f"CCL_Size_{size}.png", dpi=600, bbox_inches='tight')
#     plt.show()


# # ============================================================
# # 6. MAIN
# # ============================================================

# if __name__ == "__main__":

#     image = load_grayscale_image("comb.img")
#     binary_mask = generate_binary_mask(image, 128)

#     start = time.time()
#     labeled_image = label_components_4connect(binary_mask)
#     end = time.time()

#     print(f"\nCCL Execution Time: {end-start:.4f} seconds")

#     size_filters = [500, 750, 1000]

#     for size in size_filters:

#         print("\n" + "="*70)
#         print(f"Minimum Size Threshold: {size}")
#         print("="*70)

#         components = extract_geometric_descriptors(labeled_image, size)

#         # ✅ REMOVE SMALL COMPONENTS FROM IMAGE (Strict Assignment Requirement)
#         filtered_labels = labeled_image.copy()
#         valid_labels = set(components.keys())

#         for lab in np.unique(filtered_labels):
#             if lab != 0 and lab not in valid_labels:
#                 filtered_labels[filtered_labels == lab] = 0

#         print(f"Valid Components: {len(components)}")

#         print(f"\n{'ID':<6}{'Area':<10}{'Centroid':<20}{'Eccentricity':<15}{'Compactness':<15}")
#         print("-"*75)

#         for lab, comp in components.items():

#             xc, yc = comp["centroid"]

#             print(f"{lab:<6}{comp['area']:<10}"
#                 f"({xc:.2f}, {yc:.2f})".ljust(20) +
#                 f"{comp['eccentricity']:<15.4f}"
#                 f"{comp['compactness']:<15.2f}")

#         total_area = sum([c["area"] for c in components.values()])
#         binary_area = np.sum(binary_mask)

#         print("\nEuler Number (approx):", len(components))
#         print("Area Consistency Check:", total_area <= binary_area)
#         render_feature_table(components, size)
#         render_results(image, binary_mask, filtered_labels, components, size)


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from collections import defaultdict
import time

plt.rcParams.update({
    "font.size": 11,
    "font.family": "serif",
    "axes.titlesize": 12,
    "axes.labelsize": 11
})

# ============================================================
# 1. IMAGE LOADING
# ============================================================

def load_grayscale_image(path):
    with open(path, 'rb') as f:
        # [cite: 39] Strip 512 byte header
        _ = f.read(512)
        img = np.fromfile(f, dtype=np.uint8)
    return img.reshape((512, 512))

# ============================================================
# 2. THRESHOLDING
# ============================================================

def generate_binary_mask(img, T=128):
    # [cite: 3] Threshold at T=128
    binary = (img > T).astype(np.uint8)

    # Invert if foreground dominates
    if np.sum(binary) > (binary.size // 2):
        binary = 1 - binary

    return binary

# ============================================================
# 3. CCL ALGORITHM
# ============================================================

def label_components_4connect(binary_img):
    # [cite: 4] Iterative CCL with 4-connectivity
    rows, cols = binary_img.shape
    labels = np.zeros((rows, cols), dtype=int)
    current_label = 1
    equivalences = defaultdict(set)

    # First Pass
    for i in range(rows):
        for j in range(cols):
            if binary_img[i, j] == 1:
                neighbors = []
                if i > 0 and labels[i-1, j] > 0: neighbors.append(labels[i-1, j])
                if j > 0 and labels[i, j-1] > 0: neighbors.append(labels[i, j-1])

                if not neighbors:
                    labels[i, j] = current_label
                    current_label += 1
                elif len(neighbors) == 1:
                    labels[i, j] = neighbors[0]
                else:
                    min_label = min(neighbors)
                    labels[i, j] = min_label
                    for lab in neighbors:
                        if lab != min_label:
                            equivalences[min_label].add(lab)
                            equivalences[lab].add(min_label)

    # Resolve equivalences
    label_map = {}
    visited = set()

    def dfs(label, root):
        stack = [label]
        while stack:
            l = stack.pop()
            if l not in visited:
                visited.add(l)
                label_map[l] = root
                stack.extend(equivalences[l])

    for lab in range(1, current_label):
        if lab not in visited:
            dfs(lab, lab)

    # Second Pass
    for i in range(rows):
        for j in range(cols):
            if labels[i, j] > 0:
                labels[i, j] = label_map.get(labels[i, j], labels[i, j])

    return labels

# ============================================================
# 4. FEATURE EXTRACTION
# ============================================================

def extract_geometric_descriptors(labels, min_size):
    descriptors = {}
    unique_labels = np.unique(labels)
    unique_labels = unique_labels[unique_labels != 0]

    for lab in unique_labels:
        coords = np.argwhere(labels == lab)
        area = len(coords)

        # [cite: 7] Size filter parameter
        if area < min_size:
            continue

        y, x = coords[:, 0], coords[:, 1]
        
        #  Centroid
        xc, yc = np.mean(x), np.mean(y)

        #  Bounding Box
        min_x, max_x = np.min(x), np.max(x)
        min_y, max_y = np.min(y), np.max(y)

        # Moments for Orientation
        x_prime = x - xc
        y_prime = y - yc
        a = np.sum(x_prime**2)
        b = 2 * np.sum(x_prime * y_prime)
        c = np.sum(y_prime**2)

        #  Orientation
        theta = 0.5 * np.arctan2(b, (a - c))

        term = np.sqrt((a - c)**2 + b**2)
        Imax = (a + c + term) / 2
        Imin = (a + c - term) / 2

        # [cite: 20] Eccentricity
        eccentricity = np.sqrt(1 - (Imin / Imax)) if Imax > 0 else 0

        # [cite: 20] Compactness (using perimeter approximation)
        perimeter = 0
        rows, cols = labels.shape
        for py, px in coords:
            # Simple boundary check: if any 4-neighbor is 0 or different label
            is_boundary = False
            if py==0 or py==rows-1 or px==0 or px==cols-1: is_boundary = True
            elif (labels[py-1, px]!=lab or labels[py+1, px]!=lab or 
                  labels[py, px-1]!=lab or labels[py, px+1]!=lab):
                is_boundary = True
            
            if is_boundary: perimeter += 1

        compactness = (perimeter**2) / area if area > 0 else 0

        descriptors[lab] = {
            "area": area,
            "centroid": (xc, yc),
            "bbox": (min_x, min_y, max_x, max_y),
            "orientation": theta,
            "eccentricity": eccentricity,
            "compactness": compactness
        }

    return descriptors

# ============================================================
# 5. VISUALIZATION (RESTORED TABLE & OVERLAYS)
# ============================================================

def render_feature_table(descriptors, size):
    """
    Restored function to save the features as a PNG table.
    """
    table_data = []
    for lab, comp in descriptors.items():
        xc, yc = comp["centroid"]
        table_data.append([
            lab,
            comp["area"],
            f"({xc:.1f}, {yc:.1f})",
            f"{comp['eccentricity']:.3f}",
            f"{comp['compactness']:.2f}"
        ])

    column_labels = ["ID", "Area", "Centroid", "Eccentricity", "Compactness"]

    fig, ax = plt.subplots(figsize=(10, len(table_data) * 0.5 + 2))
    ax.axis('off')
    
    table = ax.table(
        cellText=table_data,
        colLabels=column_labels,
        loc='center',
        cellLoc='center'
    )
    table.auto_set_font_size(False)
    table.set_fontsize(10)
    table.scale(1, 1.5)
    
    plt.title(f"Component Features (Size >= {size})", pad=20)
    plt.tight_layout()
    plt.savefig(f"Table_Size_{size}.png", dpi=150, bbox_inches='tight')
    plt.close()
    print(f"Saved Table_Size_{size}.png")

def save_image_C(filtered_labels, descriptors, size):
    """
    Saves 'Image C' with required overlays:
    - [cite: 21] Unique colors for components
    -  Overlaid computed features
    """
    fig, ax = plt.subplots(figsize=(8, 8))
    
    colored_img = np.zeros((*filtered_labels.shape, 3))
    np.random.seed(42) 
    for lab in descriptors.keys():
        color = np.random.rand(3)
        while np.mean(color) < 0.3: color = np.random.rand(3) # Avoid dark colors
        colored_img[filtered_labels == lab] = color

    ax.imshow(colored_img)
    ax.set_title(f"Image C (Size >= {size})")
    ax.axis('off')

    for lab, comp in descriptors.items():
        xc, yc = comp["centroid"]
        min_x, min_y, max_x, max_y = comp["bbox"]

        # Overlay Bounding Box
        rect = patches.Rectangle((min_x, min_y), max_x - min_x, max_y - min_y,
                                 linewidth=1, edgecolor='white', facecolor='none', linestyle='--')
        ax.add_patch(rect)

        # Overlay Orientation Axis
        length = 30
        theta = comp["orientation"]
        x2 = xc + length * np.cos(theta)
        y2 = yc + length * np.sin(theta)
        ax.plot([xc, x2], [yc, y2], color='white', linewidth=1.5)

        # Overlay ID (at Centroid)
        ax.text(xc, yc, str(lab), color='red', fontsize=9, fontweight='bold',
                ha='center', va='center', bbox=dict(facecolor='white', alpha=0.7, pad=0.5))

    plt.tight_layout()
    plt.savefig(f"Image_C_Size_{size}.png", dpi=150, bbox_inches='tight')
    plt.close()
    print(f"Saved Image_C_Size_{size}.png")

# ============================================================
# 6. MAIN EXECUTION
# ============================================================

if __name__ == "__main__":
    image_path = "comb.img"
    try:
        image = load_grayscale_image(image_path)
    except FileNotFoundError:
        print("Error: comb.img not found.")
        exit()

    # Save B
    plt.imsave("Image_B.png", image, cmap='gray')
    
    # Save BT
    binary_mask = generate_binary_mask(image, 128)
    plt.imsave("Image_BT.png", binary_mask, cmap='gray')

    # Run CCL
    labeled_image = label_components_4connect(binary_mask)

    # Process sizes
    size_filters = [100, 500, 1000] # Adjust as needed
    for size in size_filters:
        print(f"Processing size {size}...")
        
        # Extract & Filter
        components = extract_geometric_descriptors(labeled_image, size)
        
        # Filter label map for visualization
        filtered_labels = np.zeros_like(labeled_image)
        for lab in components.keys():
            filtered_labels[labeled_image == lab] = lab

        # Save Table Image
        render_feature_table(components, size)
        
        # Save Component Image with Overlays
        save_image_C(filtered_labels, components, size)

    print("Done.")

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from collections import defaultdict
import time

# ============================================================
# CONFIGURATION & STYLE
# ============================================================
plt.rcParams.update({
    "font.family": "serif",
    "font.size": 10,
    "axes.titlesize": 12,
    "axes.labelsize": 10,
    "figure.dpi": 150
})

# ============================================================
# PIPELINE STEP 1: PREPROCESSING
# ============================================================

def load_and_preprocess(path):
    """Loads raw image, strips header, returns original and binary."""
    try:
        with open(path, 'rb') as f:
            _ = f.read(512) # Strip header
            img = np.fromfile(f, dtype=np.uint8)
        
        original = img.reshape((512, 512))
        
        # Thresholding (T=128)
        binary = (original > 128).astype(np.uint8)
        
        # Automatic foreground detection (invert if foreground is majority)
        if np.sum(binary) > (binary.size // 2):
            binary = 1 - binary
            
        return original, binary
    except FileNotFoundError:
        print(f"Error: File {path} not found.")
        exit()

# ============================================================
# PIPELINE STEP 2: SEGMENTATION (CCL ALGORITHM)
# ============================================================

def label_components(binary_img):
    """Iterative CCL with 4-connectivity."""
    rows, cols = binary_img.shape
    labels = np.zeros((rows, cols), dtype=int)
    current_label = 1
    equivalences = defaultdict(set)

    # Pass 1: Assign temporary labels
    for i in range(rows):
        for j in range(cols):
            if binary_img[i, j] == 1:
                neighbors = []
                if i > 0 and labels[i-1, j] > 0: neighbors.append(labels[i-1, j])
                if j > 0 and labels[i, j-1] > 0: neighbors.append(labels[i, j-1])

                if not neighbors:
                    labels[i, j] = current_label
                    current_label += 1
                elif len(neighbors) == 1:
                    labels[i, j] = neighbors[0]
                else:
                    min_lab = min(neighbors)
                    labels[i, j] = min_lab
                    for lab in neighbors:
                        if lab != min_lab:
                            equivalences[min_lab].add(lab)
                            equivalences[lab].add(min_lab)

    # Resolve Equivalences
    label_map = {}
    visited = set()
    
    def dfs(node, root):
        stack = [node]
        while stack:
            n = stack.pop()
            if n not in visited:
                visited.add(n)
                label_map[n] = root
                stack.extend(equivalences[n])

    for i in range(1, current_label):
        if i not in visited:
            dfs(i, i)

    # Pass 2: Apply resolved labels
    for i in range(rows):
        for j in range(cols):
            if labels[i, j] > 0:
                labels[i, j] = label_map.get(labels[i, j], labels[i, j])

    return labels

# ============================================================
# PIPELINE STEP 3: POST-PROCESSING (RE-LABELING)
# ============================================================

def filter_and_relabel(labels, min_size):
    """
    Filters small components and re-indexes valid ones to 1, 2, 3...
    Returns: Filtered label map, mapping dict (new_id -> old_id)
    """
    unique_labels, counts = np.unique(labels, return_counts=True)
    valid_mask = (unique_labels != 0) & (counts >= min_size)
    valid_labels = unique_labels[valid_mask]
    
    new_labels = np.zeros_like(labels)
    mapping = {}
    
    for new_id, old_id in enumerate(valid_labels, 1):
        new_labels[labels == old_id] = new_id
        mapping[new_id] = old_id
        
    return new_labels, mapping

# ============================================================
# PIPELINE STEP 4: FEATURE EXTRACTION & ANALYSIS
# ============================================================

def extract_features(labels):
    """Calculates geometric properties and shape classification."""
    props = {}
    indices = np.unique(labels)
    indices = indices[indices != 0]

    for lab in indices:
        coords = np.argwhere(labels == lab)
        y, x = coords[:, 0], coords[:, 1]
        area = len(coords)
        
        # Centroid
        xc, yc = np.mean(x), np.mean(y)
        
        # Bounding Box
        min_x, max_x = np.min(x), np.max(x)
        min_y, max_y = np.min(y), np.max(y)
        
        # Central Moments
        x_prime = x - xc
        y_prime = y - yc
        a = np.sum(x_prime**2)
        b = 2 * np.sum(x_prime * y_prime)
        c = np.sum(y_prime**2)
        
        # Orientation & Principal Moments
        theta = 0.5 * np.arctan2(b, a - c)
        term = np.sqrt((a - c)**2 + b**2)
        Imax = (a + c + term) / 2
        Imin = (a + c - term) / 2
        
        # Moment Validation Check (a+c should equal Imax+Imin)
        moment_error = abs((a + c) - (Imax + Imin))
        
        # Shape Descriptors
        elongation = Imax / Imin if Imin > 0 else 0
        eccentricity = np.sqrt(1 - (Imin / Imax)) if Imax > 0 else 0
        
        # Perimeter (Boundary pixel count)
        perimeter = 0
        rows, cols = labels.shape
        # Quick boundary check using erosion logic would be faster, 
        # but sticking to iteration for clarity/assignment constraints
        for py, px in coords:
            is_edge = (py==0 or py==rows-1 or px==0 or px==cols-1)
            if not is_edge:
                # Check 4-neighbors
                if (labels[py-1, px]!=lab or labels[py+1, px]!=lab or 
                    labels[py, px-1]!=lab or labels[py, px+1]!=lab):
                    is_edge = True
            if is_edge: perimeter += 1
            
        compactness = (perimeter**2) / area
        
        # Shape Classification
        if eccentricity < 0.6: shape_class = "Compact"
        elif eccentricity < 0.9: shape_class = "Oval"
        else: shape_class = "Elongated"

        props[lab] = {
            "area": area,
            "centroid": (xc, yc),
            "bbox": (min_x, min_y, max_x, max_y),
            "orientation": theta,
            "eccentricity": eccentricity,
            "compactness": compactness,
            "shape_class": shape_class,
            "moment_error": moment_error,
            "principal_axes": (Imax, Imin)
        }
        
    return props

# ============================================================
# PIPELINE STEP 5: VISUALIZATION & REPORTING
# ============================================================

def visualize_results(original, labels, props, size_thresh):
    """
    Generates:
    1. Image C: Labeled components with Principal Axes overlay.
    2. Histograms: Distribution of features.
    """
    
    # --- 1. Main Visualization (Image C) ---
    fig, ax = plt.subplots(figsize=(10, 10))
    
    # Create colored map
    colored = np.zeros((*labels.shape, 3))
    np.random.seed(42) # Deterministic colors
    
    # Black background
    colored[:] = [0, 0, 0]
    
    unique_labs = list(props.keys())
    for lab in unique_labs:
        color = np.random.rand(3)
        # Boost brightness/saturation
        while np.mean(color) < 0.4: color = np.random.rand(3)
        colored[labels == lab] = color

    ax.imshow(colored)
    ax.set_title(f"Image C: Filtered Components (Size ≥ {size_thresh})\n"
                 f"Visualization includes Centroids, Bounding Boxes, and Principal Axes", pad=15)
    ax.axis('off')

    for lab, p in props.items():
        xc, yc = p["centroid"]
        min_x, min_y, max_x, max_y = p["bbox"]
        theta = p["orientation"]
        
        # Bounding Box (Subtle)
        rect = patches.Rectangle((min_x, min_y), max_x-min_x, max_y-min_y,
                                 linewidth=0.8, edgecolor='white', facecolor='none', 
                                 linestyle=':', alpha=0.7)
        ax.add_patch(rect)
        
        # Principal Axes Calculation
        # Length scaled for visibility (not exact eigen-length)
        axis_len = 30 
        
        # Major Axis (Yellow)
        x1 = xc + axis_len * np.cos(theta)
        y1 = yc + axis_len * np.sin(theta)
        x2 = xc - axis_len * np.cos(theta)
        y2 = yc - axis_len * np.sin(theta)
        ax.plot([x1, x2], [y1, y2], color='yellow', linewidth=1.5, alpha=0.9)
        
        # Minor Axis (Red) - Perpendicular
        x3 = xc + (axis_len * 0.6) * np.cos(theta + np.pi/2)
        y3 = yc + (axis_len * 0.6) * np.sin(theta + np.pi/2)
        x4 = xc - (axis_len * 0.6) * np.cos(theta + np.pi/2)
        y4 = yc - (axis_len * 0.6) * np.sin(theta + np.pi/2)
        ax.plot([x3, x4], [y3, y4], color='red', linewidth=1.5, alpha=0.9)

        # Label
        ax.text(xc, yc, str(lab), color='cyan', fontsize=8, fontweight='bold',
                ha='center', va='center', bbox=dict(facecolor='black', alpha=0.5, pad=0))

    # Legend for Axes
    legend_elements = [
        plt.Line2D([0], [0], color='yellow', lw=2, label='Major Axis'),
        plt.Line2D([0], [0], color='red', lw=2, label='Minor Axis')
    ]
    ax.legend(handles=legend_elements, loc='upper right', fontsize=8)

    outfile = f"CCL_Result_Size_{size_thresh}.png"
    plt.tight_layout()
    plt.savefig(outfile, bbox_inches='tight', dpi=200)
    plt.close()
    print(f"[Saved] {outfile}")

    # --- 2. Statistical Histograms ---
    if len(props) > 1:
        fig_hist, axes = plt.subplots(1, 3, figsize=(15, 4))
        
        areas = [p['area'] for p in props.values()]
        eccs = [p['eccentricity'] for p in props.values()]
        comps = [p['compactness'] for p in props.values()]
        
        # Area Hist
        axes[0].hist(areas, bins=10, color='skyblue', edgecolor='black')
        axes[0].set_title("Area Distribution")
        axes[0].set_xlabel("Pixels")
        
        # Eccentricity Hist
        axes[1].hist(eccs, bins=10, range=(0,1), color='salmon', edgecolor='black')
        axes[1].set_title("Eccentricity Distribution")
        axes[1].set_xlabel("0 (Circle) -> 1 (Line)")
        
        # Compactness Hist
        axes[2].hist(comps, bins=10, color='lightgreen', edgecolor='black')
        axes[2].set_title("Compactness Distribution")
        axes[2].set_xlabel("P^2 / Area")
        
        fig_hist.suptitle(f"Feature Statistics (Size ≥ {size_thresh})", fontsize=14)
        hist_file = f"CCL_Histograms_Size_{size_thresh}.png"
        plt.savefig(hist_file, bbox_inches='tight')
        plt.close()
        print(f"[Saved] {hist_file}")

def print_statistical_report(props, size_thresh):
    """Prints a professional summary table to console."""
    print(f"\n{'='*80}")
    print(f"ANALYSIS REPORT | Minimum Size Threshold: {size_thresh} pixels")
    print(f"{'='*80}")
    
    if not props:
        print("No components found.")
        return

    # Summary Stats
    areas = [p['area'] for p in props.values()]
    avg_area = np.mean(areas)
    avg_ecc = np.mean([p['eccentricity'] for p in props.values()])
    max_moment_error = max([p['moment_error'] for p in props.values()])
    
    print(f"Total Components: {len(props)}")
    print(f"Average Area:     {avg_area:.1f} pixels")
    print(f"Avg Eccentricity: {avg_ecc:.3f}")
    print(f"Moment Val. Err:  {max_moment_error:.1e} (Should be near 0)")
    
    print(f"\n{'ID':<4} {'Area':<8} {'Centroid':<18} {'Eccentricity':<14} {'Class':<10} {'Moment Check'}")
    print("-" * 80)
    
    for lab in sorted(props.keys()):
        p = props[lab]
        xc, yc = p['centroid']
        err_flag = "OK" if p['moment_error'] < 1e-5 else "FAIL"
        
        print(f"{lab:<4} {p['area']:<8} "
              f"({xc:>6.1f}, {yc:>6.1f})   "
              f"{p['eccentricity']:<14.4f} "
              f"{p['shape_class']:<10} "
              f"{err_flag}")
    print("-" * 80)

# ============================================================
# MAIN EXECUTION
# ============================================================

if __name__ == "__main__":
    
    print(">>> Starting Computer Vision Pipeline...")
    
    # 1. Preprocessing
    original_img, binary_mask = load_and_preprocess("comb.img")
    
    # Save Hardcopies required by assignment
    plt.imsave("01_Original_B.png", original_img, cmap='gray')
    plt.imsave("02_Binary_BT.png", binary_mask, cmap='gray')
    print("[Saved] Base images (B and BT)")

    # 2. Segmentation (CCL)
    print(">>> Running Connected Component Labeling...")
    raw_labels = label_components(binary_mask)
    
    # 3. Analysis Loop
    size_filters = [100, 500, 1000]
    
    for size in size_filters:
        # Re-label and Filter
        clean_labels, _ = filter_and_relabel(raw_labels, size)
        
        # Extract Features
        features = extract_features(clean_labels)
        
        # Generate Professional Report
        print_statistical_report(features, size)
        
        # Visualize (Image C + Histograms)
        visualize_results(original_img, clean_labels, features, size)

    print("\n>>> Pipeline Complete. All artifacts generated.")

# last time

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from collections import defaultdict
import time

# ============================================================
# CONFIGURATION & STYLE
# ============================================================
plt.rcParams.update({
    "font.family": "serif",
    "font.size": 10,
    "axes.titlesize": 12,
    "axes.labelsize": 10,
    "figure.dpi": 150
})

# ============================================================
# PIPELINE STEP 1: PREPROCESSING
# ============================================================

def load_and_preprocess(path):
    """Loads raw image, strips header, returns original and binary."""
    try:
        with open(path, 'rb') as f:
            _ = f.read(512) # Strip header
            img = np.fromfile(f, dtype=np.uint8)
        
        original = img.reshape((512, 512))
        
        # [cite_start]Thresholding (T=128) [cite: 3]
        binary = (original > 128).astype(np.uint8)
        
        # Automatic foreground detection (invert if foreground is majority)
        if np.sum(binary) > (binary.size // 2):
            binary = 1 - binary
            
        return original, binary
    except FileNotFoundError:
        print(f"Error: File {path} not found.")
        exit()

# ============================================================
# PIPELINE STEP 2: SEGMENTATION (CCL ALGORITHM)
# ============================================================

def label_components(binary_img):
    """Iterative CCL with 4-connectivity."""
    rows, cols = binary_img.shape
    labels = np.zeros((rows, cols), dtype=int)
    current_label = 1
    equivalences = defaultdict(set)

    # Pass 1: Assign temporary labels
    for i in range(rows):
        for j in range(cols):
            if binary_img[i, j] == 1:
                neighbors = []
                if i > 0 and labels[i-1, j] > 0: neighbors.append(labels[i-1, j])
                if j > 0 and labels[i, j-1] > 0: neighbors.append(labels[i, j-1])

                if not neighbors:
                    labels[i, j] = current_label
                    current_label += 1
                elif len(neighbors) == 1:
                    labels[i, j] = neighbors[0]
                else:
                    min_lab = min(neighbors)
                    labels[i, j] = min_lab
                    for lab in neighbors:
                        if lab != min_lab:
                            equivalences[min_lab].add(lab)
                            equivalences[lab].add(min_lab)

    # Resolve Equivalences
    label_map = {}
    visited = set()
    
    def dfs(node, root):
        stack = [node]
        while stack:
            n = stack.pop()
            if n not in visited:
                visited.add(n)
                label_map[n] = root
                stack.extend(equivalences[n])

    for i in range(1, current_label):
        if i not in visited:
            dfs(i, i)

    # Pass 2: Apply resolved labels
    for i in range(rows):
        for j in range(cols):
            if labels[i, j] > 0:
                labels[i, j] = label_map.get(labels[i, j], labels[i, j])

    return labels

# ============================================================
# PIPELINE STEP 3: POST-PROCESSING (RE-LABELING)
# ============================================================

def filter_and_relabel(labels, min_size):
    """
    Filters small components and re-indexes valid ones to 1, 2, 3...
    Returns: Filtered label map, mapping dict (new_id -> old_id)
    """
    unique_labels, counts = np.unique(labels, return_counts=True)
    valid_mask = (unique_labels != 0) & (counts >= min_size)
    valid_labels = unique_labels[valid_mask]
    
    new_labels = np.zeros_like(labels)
    mapping = {}
    
    for new_id, old_id in enumerate(valid_labels, 1):
        new_labels[labels == old_id] = new_id
        mapping[new_id] = old_id
        
    return new_labels, mapping

# ============================================================
# PIPELINE STEP 4: FEATURE EXTRACTION & ANALYSIS
# ============================================================

def extract_features(labels):
    """Calculates geometric properties aligned with Lecture Notes."""
    props = {}
    indices = np.unique(labels)
    indices = indices[indices != 0]

    for lab in indices:
        # np.argwhere returns (row, col)
        coords = np.argwhere(labels == lab)
        y, x = coords[:, 0], coords[:, 1] # y=row, x=col
        area = len(coords)
        
        # [cite_start]1. Centroid [cite: 14]
        # Xc = mean(columns), Yc = mean(rows)
        xc, yc = np.mean(x), np.mean(y)
        
        # [cite_start]Bounding Box [cite: 16]
        min_x, max_x = np.min(x), np.max(x)
        min_y, max_y = np.min(y), np.max(y)
        
        # 2. Central Moments
        x_prime = x - xc
        y_prime = y - yc
        
        # a = sum(x'^2), b = 2*sum(x'y'), c = sum(y'^2)
        a = np.sum(x_prime**2)
        b = 2 * np.sum(x_prime * y_prime)
        c = np.sum(y_prime**2)
        
        # 3. Orientation (Theta)
        # tan(2theta) = b / (a - c)
        theta = 0.5 * np.arctan2(b, a - c)
        
        # 4. Principal Moments (Imax, Imin)
        # Using explicit eigenvalues formulation
        term = np.sqrt((a - c)**2 + b**2)
        Imax = (a + c + term) / 2
        Imin = (a + c - term) / 2
        
        # Validation: a+c should equal Imax+Imin (Invariant)
        moment_error = abs((a + c) - (Imax + Imin))
        
        # 5. Eccentricity & Compactness
        elongation = Imax / Imin if Imin > 0 else 0
        
        # [cite_start]Eccentricity formula: sqrt(1 - Imin/Imax) [cite: 20]
        eccentricity = np.sqrt(1 - (Imin / Imax)) if Imax > 0 else 0
        
        # [cite_start]Perimeter (Boundary pixel count) [cite: 20]
        perimeter = 0
        rows, cols = labels.shape
        for py, px in coords:
            # Check 4-neighbors to see if it's an edge
            is_edge = (py==0 or py==rows-1 or px==0 or px==cols-1)
            if not is_edge:
                if (labels[py-1, px]!=lab or labels[py+1, px]!=lab or 
                    labels[py, px-1]!=lab or labels[py, px+1]!=lab):
                    is_edge = True
            if is_edge: perimeter += 1
            
        compactness = (perimeter**2) / area
        
        # 6. Shape Classification
        if eccentricity < 0.6: shape_class = "Compact"
        elif eccentricity < 0.9: shape_class = "Oval"
        else: shape_class = "Elongated"

        props[lab] = {
            "area": area,
            "centroid": (xc, yc),
            "bbox": (min_x, min_y, max_x, max_y),
            "orientation": theta,
            "eccentricity": eccentricity,
            "compactness": compactness,
            "shape_class": shape_class,
            "moment_error": moment_error,
            "principal_axes": (Imax, Imin)
        }
        
    return props

# ============================================================
# PIPELINE STEP 5: VISUALIZATION & REPORTING
# ============================================================

def save_feature_table(props, size_thresh):
    """Generates and saves the APA-style table as an image."""
    if not props:
        return

    table_data = []
    # Sort by ID
    for lab in sorted(props.keys()):
        p = props[lab]
        xc, yc = p["centroid"]
        table_data.append([
            lab,
            p["area"],
            f"({xc:.1f}, {yc:.1f})",
            f"{p['eccentricity']:.3f}",
            f"{p['compactness']:.2f}",
            p['shape_class']
        ])

    column_labels = ["ID", "Area", "Centroid", "Eccentricity", "Compactness", "Shape"]

    # Dynamic height based on row count
    fig_h = len(table_data) * 0.3 + 1.5
    fig, ax = plt.subplots(figsize=(10, fig_h))
    ax.axis('off')
    
    table = ax.table(
        cellText=table_data,
        colLabels=column_labels,
        loc='center',
        cellLoc='center'
    )
    table.auto_set_font_size(False)
    table.set_fontsize(10)
    table.scale(1, 1.5)
    
    # Styling for APA Look
    for (row, col), cell in table.get_celld().items():
        if row == 0:
            cell.set_text_props(weight='bold')
            cell.set_facecolor('#f2f2f2') # Light gray header

    plt.title(f"Table 1: Component Features (Size ≥ {size_thresh})", pad=10, fontsize=12, fontweight='bold')
    plt.tight_layout()
    outfile = f"Table_Size_{size_thresh}.png"
    plt.savefig(outfile, dpi=200, bbox_inches='tight')
    plt.close()
    print(f"[Saved] {outfile}")

def visualize_results(original, labels, props, size_thresh):
    """
    Generates Image C (with overlays) and Histograms.
    """
    
    # --- 1. Main Visualization (Image C) ---
    fig, ax = plt.subplots(figsize=(10, 10))
    
    # Create colored map
    colored = np.zeros((*labels.shape, 3))
    np.random.seed(42) # Deterministic colors
    
    # Black background
    colored[:] = [0, 0, 0]
    
    unique_labs = list(props.keys())
    for lab in unique_labs:
        color = np.random.rand(3)
        # Boost brightness/saturation
        while np.mean(color) < 0.4: color = np.random.rand(3)
        colored[labels == lab] = color

    ax.imshow(colored)
    ax.set_title(f"Image C: Filtered Components (Size ≥ {size_thresh})\n"
                 f"Visualization includes Centroids, Bounding Boxes, and Principal Axes", pad=15)
    ax.axis('off')

    for lab, p in props.items():
        xc, yc = p["centroid"]
        min_x, min_y, max_x, max_y = p["bbox"]
        theta = p["orientation"]
        
        # Bounding Box (Subtle)
        rect = patches.Rectangle((min_x, min_y), max_x-min_x, max_y-min_y,
                                 linewidth=0.8, edgecolor='white', facecolor='none', 
                                 linestyle=':', alpha=0.7)
        ax.add_patch(rect)
        
        # Principal Axes Calculation
        # Length scaled for visibility (not exact eigen-length)
        axis_len = 30 
        
        # Major Axis (Yellow)
        x1 = xc + axis_len * np.cos(theta)
        y1 = yc + axis_len * np.sin(theta)
        x2 = xc - axis_len * np.cos(theta)
        y2 = yc - axis_len * np.sin(theta)
        ax.plot([x1, x2], [y1, y2], color='yellow', linewidth=1.5, alpha=0.9)
        
        # Minor Axis (Red) - Perpendicular
        x3 = xc + (axis_len * 0.6) * np.cos(theta + np.pi/2)
        y3 = yc + (axis_len * 0.6) * np.sin(theta + np.pi/2)
        x4 = xc - (axis_len * 0.6) * np.cos(theta + np.pi/2)
        y4 = yc - (axis_len * 0.6) * np.sin(theta + np.pi/2)
        ax.plot([x3, x4], [y3, y4], color='red', linewidth=1.5, alpha=0.9)

        # Label
        ax.text(xc, yc, str(lab), color='cyan', fontsize=8, fontweight='bold',
                ha='center', va='center', bbox=dict(facecolor='black', alpha=0.5, pad=0))

    # Legend for Axes
    legend_elements = [
        plt.Line2D([0], [0], color='yellow', lw=2, label='Major Axis'),
        plt.Line2D([0], [0], color='red', lw=2, label='Minor Axis')
    ]
    ax.legend(handles=legend_elements, loc='upper right', fontsize=8)

    outfile = f"Image_C_Size_{size_thresh}.png"
    plt.tight_layout()
    plt.savefig(outfile, bbox_inches='tight', dpi=200)
    plt.close()
    print(f"[Saved] {outfile}")

    # --- 2. Statistical Histograms ---
    if len(props) > 1:
        fig_hist, axes = plt.subplots(1, 3, figsize=(15, 4))
        
        areas = [p['area'] for p in props.values()]
        eccs = [p['eccentricity'] for p in props.values()]
        comps = [p['compactness'] for p in props.values()]
        
        # Area Hist
        axes[0].hist(areas, bins=10, color='skyblue', edgecolor='black')
        axes[0].set_title("Area Distribution")
        axes[0].set_xlabel("Pixels")
        
        # Eccentricity Hist
        axes[1].hist(eccs, bins=10, range=(0,1), color='salmon', edgecolor='black')
        axes[1].set_title("Eccentricity Distribution")
        axes[1].set_xlabel("0 (Circle) -> 1 (Line)")
        
        # Compactness Hist
        axes[2].hist(comps, bins=10, color='lightgreen', edgecolor='black')
        axes[2].set_title("Compactness Distribution")
        axes[2].set_xlabel("P^2 / Area")
        
        fig_hist.suptitle(f"Feature Statistics (Size ≥ {size_thresh})", fontsize=14)
        hist_file = f"Histograms_Size_{size_thresh}.png"
        plt.savefig(hist_file, bbox_inches='tight')
        plt.close()
        print(f"[Saved] {hist_file}")

def print_statistical_report(props, size_thresh):
    """Prints a professional summary table to console."""
    print(f"\n{'='*80}")
    print(f"ANALYSIS REPORT | Minimum Size Threshold: {size_thresh} pixels")
    print(f"{'='*80}")
    
    if not props:
        print("No components found.")
        return

    # Summary Stats
    areas = [p['area'] for p in props.values()]
    avg_area = np.mean(areas)
    avg_ecc = np.mean([p['eccentricity'] for p in props.values()])
    max_moment_error = max([p['moment_error'] for p in props.values()])
    
    print(f"Total Components: {len(props)}")
    print(f"Average Area:     {avg_area:.1f} pixels")
    print(f"Avg Eccentricity: {avg_ecc:.3f}")
    print(f"Moment Val. Err:  {max_moment_error:.1e} (Should be near 0)")
    
    print(f"\n{'ID':<4} {'Area':<8} {'Centroid':<18} {'Eccentricity':<14} {'Class':<10} {'Moment Check'}")
    print("-" * 80)
    
    for lab in sorted(props.keys()):
        p = props[lab]
        xc, yc = p['centroid']
        err_flag = "OK" if p['moment_error'] < 1e-5 else "FAIL"
        
        print(f"{lab:<4} {p['area']:<8} "
              f"({xc:>6.1f}, {yc:>6.1f})   "
              f"{p['eccentricity']:<14.4f} "
              f"{p['shape_class']:<10} "
              f"{err_flag}")
    print("-" * 80)

# ============================================================
# MAIN EXECUTION
# ============================================================

if __name__ == "__main__":
    
    print(">>> Starting Computer Vision Pipeline...")
    
    # 1. Preprocessing
    original_img, binary_mask = load_and_preprocess("comb.img")
    
    # [cite_start]Save Hardcopies required by assignment [cite: 26]
    plt.imsave("Image_B_Original.png", original_img, cmap='gray')
    plt.imsave("Image_BT_Binary.png", binary_mask, cmap='gray')
    print("[Saved] Image_B_Original.png and Image_BT_Binary.png")

    # 2. Segmentation (CCL)
    print(">>> Running Connected Component Labeling...")
    raw_labels = label_components(binary_mask)
    
    # 3. Analysis Loop
    size_filters = [100, 500, 1000]
    
    for size in size_filters:
        # Re-label and Filter
        clean_labels, _ = filter_and_relabel(raw_labels, size)
        
        # Extract Features
        features = extract_features(clean_labels)
        
        # Generate Professional Report (Console)
        print_statistical_report(features, size)
        
        # Generate Table Image (APA Style) - RESTORED
        save_feature_table(features, size)
        
        # Visualize (Image C + Histograms)
        visualize_results(original_img, clean_labels, features, size)

    print("\n>>> Pipeline Complete. All artifacts generated.")

>>> Starting Computer Vision Pipeline...
[Saved] Image_B_Original.png and Image_BT_Binary.png
>>> Running Connected Component Labeling...

ANALYSIS REPORT | Minimum Size Threshold: 100 pixels
Total Components: 7
Average Area:     10908.7 pixels
Avg Eccentricity: 0.831
Moment Val. Err:  6.0e-08 (Should be near 0)

ID   Area     Centroid           Eccentricity   Class      Moment Check
--------------------------------------------------------------------------------
1    620      ( 506.4,   28.2)   0.9912         Elongated  OK
2    13148    ( 207.3,   91.7)   0.3883         Compact    OK
3    19762    ( 361.1,  146.6)   0.9053         Elongated  OK
4    15327    (  81.3,  196.6)   0.9631         Elongated  OK
5    119      ( 511.0,  175.0)   1.0000         Elongated  OK
6    8913     ( 207.2,  269.8)   0.5731         Compact    OK
7    18472    ( 241.1,  493.0)   0.9968         Elongated  OK
--------------------------------------------------------------------------------
[Saved] Table_Siz